In [12]:
from processor.processor import Processor as processor
from database.adatabase import ADatabase
from xgboost import XGBRegressor
from statistics import mean
import math
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm
from dotenv import load_dotenv
from asset.stock import Stock
from asset.bond import Bond
from asset.option import Option
import warnings
import pytz
import copy
import pickle
warnings.simplefilter(action="ignore")

In [13]:
factors =  [
            "assets"
            ,"liabilities"
            ,"netincomeloss"
            ,"adjclose" 
            ,"rf"
            ,"spy"
           ]
required = ["year","quarter","ticker"]
required.extend(factors)
market = ADatabase("market")
sec = ADatabase("sec")
market = ADatabase("market")
fred = ADatabase("fred")

In [14]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",attrs={"id":"constituents"})[0].rename(columns={"Symbol":"ticker"})

In [15]:
fred.connect()
market_yield = fred.retrieve("market_yield")
market_yield = market_yield.rename(columns={"value":"rf"})
market_yield["rf"] = market_yield["rf"].replace(".",np.nan)
market_yield.dropna(inplace=True)
market_yield["rf"] = [float(x)/100 for x in market_yield["rf"]]
market_yield["date"] = market_yield["date"].shift(-5)
market_yield = processor.column_date_processing(market_yield)
spy = fred.retrieve("sp500")
spy = spy.rename(columns={"value":"spy"})
spy["spy"] = spy["spy"].replace(".",np.nan)
spy.dropna(inplace=True)
spy["spy"] = [float(x) for x in spy["spy"]]
spy = processor.column_date_processing(spy)
fred.disconnect()

In [16]:
data = []
sec.connect()
market.connect()
for ticker in tqdm(sp500["ticker"]):
    try:
        cik = int(sp500[sp500["ticker"]==ticker]["CIK"].item())
        filing = sec.query("filings",{"cik":cik}).drop("date",axis=1)
        prices = processor.column_date_processing(market.query("prices",{"ticker":ticker})).drop("date",axis=1)
        filing["ticker"] = ticker
        filing["year"] = filing["year"] + 1
        ticker_data = prices.merge(filing,on=["year","quarter","ticker"],how="left")
        ticker_data = ticker_data.merge(market_yield[["year","quarter","rf"]].groupby(["year","quarter"]).mean().reset_index(),on=["year","quarter"],how="left")
        ticker_data = ticker_data.merge(spy[["year","quarter","spy"]].groupby(["year","quarter"]).mean().reset_index(),on=["year","quarter"],how="left")
        ticker_data = ticker_data.groupby(["year","quarter","ticker"]).mean().reset_index()
        ticker_data.sort_values(["year","quarter"],inplace=True)
        data.append(ticker_data.bfill().ffill().dropna()[required])
    except Exception as e:
        print(ticker,str(e))
        continue
sec.disconnect()
market.disconnect()

 13%|██████████████████▉                                                                                                                                  | 64/503 [00:06<00:37, 11.61it/s]

BRK.B 'date'


 16%|███████████████████████▉                                                                                                                             | 81/503 [00:07<00:37, 11.32it/s]

BF.B 'date'


 43%|███████████████████████████████████████████████████████████████▎                                                                                    | 215/503 [00:20<00:20, 13.99it/s]

GEV "['date'] not found in axis"


 84%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                       | 422/503 [00:41<00:08, 10.06it/s]

SOLV "['date'] not found in axis"


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 503/503 [00:49<00:00, 10.13it/s]

ZTS 'date'


In [17]:
training_data = pd.concat(data).sort_values(["year","quarter"]).merge(sp500[["ticker","GICS Sector"]],on="ticker")

In [18]:
sim = training_data[training_data["year"]>=2022]
db = ADatabase("sapling")
db.cloud_connect()
model_df = db.retrieve("model")
db.disconnect()
model = pickle.loads(model_df["model"].iloc[0])
sim["prediction"] = model.predict(sim[factors])

In [19]:
sim[["year","quarter","ticker","GICS Sector","prediction"]]

,year,quarter,ticker,GICS Sector,prediction
22866,2022,1,MMM,Industrials,114.215393
22867,2022,1,AOS,Industrials,57.421696
22868,2022,1,ADBE,Information Technology,415.199799
22869,2022,1,AES,Utilities,20.313551
22870,2022,1,AFL,Financials,58.080116
...,...,...,...,...,...
26867,2024,3,WYNN,Consumer Discretionary,86.167366
26868,2024,3,XYL,Industrials,123.065758
26869,2024,3,YUM,Consumer Discretionary,128.127625
26870,2024,3,ZBRA,Information Technology,307.328491


In [20]:
db.cloud_connect()
db.drop("sim")
db.store("sim",sim[["year","quarter","ticker","GICS Sector","prediction"]])
db.disconnect()